<h1><center>Using Nextflow on OSCAR: Basic Bioinformatics Workflows</center></h1>
<p><center>Instructors: Jordan Lawson and Paul Cao</center>
 <center>Center for Computation and Visualization</center>
 <center>Center for Computational Biology of Human Disease - Computational Biology Core</center></p>

Resources for help @brown <br> 

COBRE CBHD Computational Biology Core
- Office hours
- https://cbc.brown.edu
- slack channel on ccv-share
- cbc-help@brown.edu <br>

Center for Computation and Visualization
- Office hours
- https://ccv.brown.edu
- ccv-share slack channel
- https://docs.ccv.brown.edu
- support@ccv.brown.edu

## What is Covered in This Tutorial?  

In a previous workshop, we went over simple "Hello World" and a toy workflow. In this workshop, we will go over: 

- How to understand and run a `nf-core` pre-built workflow on OSCAR
- How to build or extend a Docker container and run it in a Nextflow pipeline
- More advanced Nextflow pipelining tactics such as:
    - Conditionals and optional parameters
    - Parsing a samplesheet of a large experiment
    - Asynchronous and synchronous job execution
    - Putting it all together in an simple RNASeq example

<img src="./img/nextflow.png" width="500"/>

### Step 1: The Setup (for folks who haven't set up their `nextflow_start` script previously):

Let's first discuss setting up our environment on OSCAR so that we can get Nextflow up and running. 

**At this point, I am going to open my terminal on Open OnDemand (OOD) so that I can walk you through and show you how each of these steps and files below look. Feel free to open your terminal as well and follow along. To do so, you can go to OOD at https://ood.ccv.brown.edu and under the Clusters tab at the top select the >_OSCAR Shell Access option. All files used today can be found on GitHub in the folder at: https://github.com/compbiocore/workflows_on_OSCAR**

#### Step 1a: In OOD terminal, set up Nextflow Configuration Script using `compbiocore/workflows_on_OSCAR`:

```bash
[pcao5@node1322 ~]$ cd ~/
[pcao5@node1322 ~]$ git clone https://github.com/compbiocore/workflows_on_OSCAR.git
[pcao5@node1322 ~]$ git clone https://github.com/compbiocore/workflows_workshop.git
```

#### Step 1b: Install `compbiocore/workflows_on_OSCAR` package:

```bash
bash ~/workflows_on_OSCAR/install_me/install.sh && source ~/.bashrc
```


For the 1st installation prompt, input `NextFlow`:

```bash
Welcome to a program designed to help you easily set up and run workflow management systems on OSCAR!

Please type which software you wish to use: Nextflow or Snakemake? Nextflow
```

For the 2nd installation prompt, input your GitHub username (e.g., `paulcao-brown`):

```bash
Nextflow software detected, initializing configuration...
What is your GitHub user name? paulcao-brown
What is your GitHub token (we will keep this secret) - [Hit Enter when Done]?
```

#### Step 1c: Create a new GitHub Token and enter it:
<img src="https://i.imgur.com/GBGDQhY.png"/>

#### Step 1d: Complete the Installation 


```bash
Currently the Nextflow default for HPC resources is: memory = 5.GB time = 2.h cpus = 2 
Do you want to change these default resources for your Nextflow pipeline [Yes or No]? No
Keeping defaults!

OUTPUT MESSAGE:

                ******************************************************************
                 NEXTFLOW is now set up and configured and ready to run on OSCAR!
                ******************************************************************
                

Your default resources for Nextflow are: memory = 5.GB time = 2.h cpus = 2 


                To further customize your pipeline for efficiency, you can enter the following 
                label '<LabelName>' options right within processes in your Nextflow .nf script:
                1. label 'OSCAR_small_job' (comes with memory = 4 GB, time = 1 hour, cpus = 1)
                2. label 'OSCAR_medium_job' (comes with memory = 8 GB, time = 16 hours, cpus = 2)
                3. label 'OSCAR_large_job' (comes with memory = 16 GB, time = 24 hours, cpus = 2)
                

README:

Please see https://github.com/compbiocore/workflows_on_OSCAR for further details on how to add the above label options to your workflow.

Note the setup is designed such that pipelines downloaded from nf-core with their own resource specs within the .nf script will override your defaults.

To run Nextflow commands, you must first type and run the nextflow_start command.

To further learn how to easily run your Nextflow pipelines on OSCAR, use the Nextflow template shell script located in your ~/nextflow_setup directory.
```

### Step 2: Running a pre-existing `nf-core` Pipeline

In the previous Nextflow workshop, we built a toy pipeline from scratch. However, sometimes we want to run more complicated workflows where we adapt/copy an existing Bioinformatics pipeline that the `nf-core` community has open-sourced instead of re-inventing the wheel. 

For this exercise, we are once again using the RRBS example that we started with, so we need a pipeline that is appropriate to use for processing RRBS data. Heading over to https://nf-co.re/pipelines we can see that the **methylseq** pipeline will work for our data. We can view the details of this pipeline, such as all of its arguments and the steps it performs, by cliking on its link or visiting here: https://nf-co.re/methylseq 

#### Step 2a. Running methylSeq

Virtually every one of `nf-core`'s workflows have a test functionality that allow us to explore how the protocol works without the hassle of setting up our own test samplesheets/inputs. 


#####  Running methylSeq test-case using `--profile test, singularity`

```bash
nextflow run nf-core/methylseq -profile test,singularity --outdir methylseq_out
```

#####  Example output:

![](https://i.imgur.com/UGXkZAZ.png)

#### Step 2b. Inspecting the Test Workflow Results:

Once a test workflow has finished running, the `nf-core` pipeline willproduce a folder labeled `$OUT_DIR/pipeline_info`. In here, you will find: 
- Execution report 
    - The flow of every step of the test pipeline that was run
    - The exact commands of each step
- samplesheet.valid.csv (the test sample sheet)
- software_version.yml (the exact versions of the software used)

##### Example of a `pipeline_info` directory on nf-core: https://nf-co.re/methylseq/results#methylseq/results-93bc5811603c287c766a0ff7e03b5b41f4483895/bismark/pipeline_info/pipeline_dag_2022-12-17_00-46-10.html

##### Trace Log: 
![](https://i.imgur.com/YsjYMNe.png)

##### Audit Log of Every Step (with Command):
![](https://i.imgur.com/qdJw4qT.png)

#### Step 2b. Modeling this Test Run and Extracting its Test Inputs and Samplesheet: 

Although we have run this test workflow and can audit the steps, we still would like to know what were the parameters that were used to run this test case. We can do so by looking at the `conf/test.config` of every `nf-core` workflow: 

##### Inspect the `conf/test.config`:

https://github.com/nf-core/methylseq/blob/master/conf/test.config: 
```bash
    // Input data
    input = "$test_data_base/samplesheet/samplesheet_test.csv"

    // Genome references
    fasta = "$test_data_base/reference/genome.fa"
    fasta_index = "$test_data_base/reference/genome.fa.fai"
```

##### Replicate the same run (but with our own local files and specifying the exact parameters):

From here, we can `wget` all of test input files and understand that the necessary parameters that were run `--fasta`, `--fasta_index` and `--input`.

```bash
#curl these files so we can use the command and original inputs locally as templates to run our workflow
wget https://github.com/nf-core/test-datasets/tree/methylseq/samplesheet/samplesheet_test.csv
wget https://github.com/nf-core/test-datasets/tree/methylseq/reference/genome.fa
wget https://github.com/nf-core/test-datasets/tree/methylseq/reference/genome.fa.fai

nextflow run nf-core/methylseq --input samplesheet_test.csv --fasta genome.fa --fasta_index genome.fa.fai --outdir methylseq_out2
```

##### View the Samplesheet:

From this samplesheet, we can now understand exactly how to structure our real potential reads for our next real run.

https://github.com/nf-core/test-datasets/tree/methylseq/samplesheet/samplesheet_test.csv:
![](https://i.imgur.com/JHFvh3B.png)

##### Step 2c. Read the `nf-core` Pipeline Documentation

We have audited the test case pipeline, its inputs and samplesheets. However, there still might be additional test-cases and parameters we would like to know. We can find all of them out on the well-documented page of each `nf-core`'s pipeline. 

Drawing on the current `methylseq` example, let's take a look here: https://nf-co.re/methylseq/2.3.0/parameters

### Step 3. Building or Adapting Your Own Docker Container

At times it may be necessary to either build your own Docker containers or adapt existing Docker/Singularity containers when running your workflow. The following sections will demonstrate how to build or adapt existing containers into your Nextflow workflow.

#### Build a Docker Container from scratch

Suppose you might like to use an alternative reads aligner, `salmon`; which is not available on OSCAR and so you would like to build a docker container so you can run it on OSCAR. The steps to do so would be as follows: 

###### 3a. First make a `Dockerfile`:
```bash
FROM ubuntu:lunar

RUN apt-get update && apt-get install -y curl cowsay
RUN curl -sSL https://github.com/COMBINE-lab/salmon/releases/download/v1.5.2/salmon-1.5.2_linux_x86_64.tar.gz | tar xz \
&& mv /salmon-*/bin/* /usr/bin/ \
&& mv /salmon-*/lib/* /usr/lib/
```

###### 3b. Build the Docker image (targetting `linux/amd64` for OSCAR and tagged as `salmon:latest`):
Explicitly specifying the Docker build platform of `linux/amd64` is necessary to make your container compatible with OSCAR (a Scientific Linux environment) versus your local environment which may or may not be compatible (e.g., MacOS ARM64 or Windows PC)./

```bash
docker build -f salmon_Dockerfile -t salmon:latest --platform linux/amd64 .
```

###### 3c. Upload to your DockerHub account:
Uploading your Docker container to the cloud then enables Nextflow running on OSCAR to pull down your container from `Dockerhub` or other container registries and use them. 

```bash
docker tag salmon:latest $DOCKER_USER/salmon:latest
docker push $DOCKER_USER/salmon:latest
```

###### Example Output:
![](https://i.imgur.com/OYPbYSs.png)

###### 3d. Reference your container with its `DockerHub` id and tag in workflow
This will instruct the Nextflow pipeline to automatically pull down your Docker container and convert them into a Singularity image (which is a safer/more compatible container that `OSCAR` prefers to run):


###### salmon.nf:
```bash
process salmon {
    container 'cowmoo/salmon:latest'

    output:
     stdout

    script:
    """
    salmon -h
    """
}

workflow {
  salmon() | view
}
```

###### run salmon.nf:

```bash
nextflow run salmon.nf
```

###### Example output that confirms Salmon container now works on `OSCAR`:
![](https://i.imgur.com/Szv8yq5.png)


###### 3e. Leverage and Use Pre-Built Docker Containers in `nf-core`:


We don't have to always build containers from scratch. We can also refer to and draw on the rich repository of `nf-core` or `biocontainers`. For example, we can find the Singualarity image url for every process defined in a `nf-core` workflow (they are always stored in `modules/nf-core/$PROCESS`). We will use `bismark` in `nf-core/methylseq` as an example:

##### Find the `bismark` process definition: 


###### https://github.com/nf-core/methylseq/blob/master/modules/nf-core/bismark/align/main.nf:
```bash
process BISMARK_ALIGN {
    container "${ workflow.containerEngine == 'singularity' && !task.ext.singularity_pull_docker_container ?
        'https://depot.galaxyproject.org/singularity/bismark:0.24.0--hdfd78af_0' :
        'quay.io/biocontainers/bismark:0.24.0--hdfd78af_0' }"
    
    """
    bismark \\
        $fastq \\
        --genome $index \\
        --bam \\
        $args
    cat <<-END_VERSIONS > versions.yml
    "${task.process}":
        bismark: \$(echo \$(bismark -v 2>&1) | sed 's/^.*Bismark Version: v//; s/Copyright.*\$//')
    END_VERSIONS
    """
    ...
}
```

##### Copy it into our workflow:

We can simply grab the singularity container url: `'https://depot.galaxyproject.org/singularity/bismark:0.24.0--hdfd78af_0'`

```bash
process BISMARK_ALIGN {
    container 'https://depot.galaxyproject.org/singularity/bismark:0.24.0--hdfd78af_0'

    output:
     stdout

    script:
    """
    bismark
    """
}
```

### Step 4. Advanced Nextflow Patterns:

In the previous section, we examined a basic Nextflow workflow and the tools you use with it, such as containers. However, this example was quite rudimentary; for more complex workflows we may not be able to get by with just simply chaining together simple tasks. For example, to utilize more complicated workflows, we may have to leverage: 

- Conditionals and optional parameters
- Parsing a samplesheet of a large experiment
- Asynchronous and synchronous job execution (or scatter-gather pattern)

We will learn all of these patterns with some toy examples and then apply and put all of the concepts together in a small RNASeq example workflow demonstration.

#### Step 4a. Refresher using the Word Count Example

Below is simply the `count_words.nf` we worked with in our previous Nextflow workshop where we chain two tasks together: `sayHello` and direct its output (e.g., "Hello Blueno") to `countWords` to count the number of words in that output. 

###### count_words.nf
```bash
#!/usr/bin/env nextflow
nextflow.enable.dsl=2 

params.name = "World"

process sayHello {
  input: 
    val name
  output:
    path "hello.txt"
  script:
    """
    echo 'Hello ${name}!' > hello.txt
    """
}

process countWords {
  input: 
    path(file_in)
  output:
    stdout
  
  script:
   """
   cat ${file_in}
   wc -w ${file_in} | awk '{print \$1}'
   """ 
}

workflow {
  countWords(sayHello(params.name)) | view
}
```

#### Step 4b. Conditional Logic

Below we want to demonstrate how conditional logic works in Nextflow workflows. We will extend the previous `count_words.nf` to have two boolean flags in our pipline `--reverse` (reverse the words in our final output) and `--count_letters` (if the flag is set, count the letters insteads of words in the final output)

##### Add `reverse` function in the body of the workflow:
```bash
params.reverse = false

workflow {
  hello_result = null

  if (params.reverse) {
    hello_result = reverse(sayHello(params.name))
  } else {
    hello_result = sayHello(params.name)
  }

  count(hello_result) | view
}
```

##### Add `count_letters` function in the function of `count`:
```bash
params.count_letters = false
... 

process count {
  input: 
    path(file_in)
  output:
    stdout
  
  script:
   if (params.count_letters) {
    """
    cat ${file_in}
    wc -m ${file_in} | awk '{print \$1}'
    """
   } else {
    """
    cat ${file_in}
    wc -w ${file_in} | awk '{print \$1}'
    """
   }
}
```

##### Example Outputs:
```bash
nextflow run workflows/count_conditional.nf

Hello World!
2

nextflow run workflows/count_conditional.nf --reverse
!dlroW olleH
2

nextflow run workflows/count_conditional.nf --reverse --count_letters
!dlroW olleH
13
```

#### Step 4c. Parsing a Sample Sheet

In real production Bioinformatics workflows, most of the time we cannot just get by with specifying a list of fasta raw or aligned reads. We also need to carry through all of the important metadata associated with a particular sample's sequencing data and provide their contexts to the analysis programs (e.g., Differential Gene Analysis where we need to differentiate between `treatment` and `control` groups; temporal analysis where we differentiate between the samples taken at different time points). 

We will demonstrate this concept by making a toy workflow that takes a samplesheet of documents and, according to a specification that we set for each document, counts either the number of letters or words of the document (determined by `count_letters` column).

##### Make a simple samplesheet:
Our toy sample sheet: 

```bash
document,count_letters
samplesheet/hello1.txt,F
samplesheet/hello2.txt,T
```

##### Make a `documents_ch` as a channel of documents; parsed from the samplesheet above:

Using `Channel.fromPath(params.samplesheet).splitCsv(header:true)`, We create a "channel" (or a "stream") of documents; where each entry is a tuple of both the document file itself and a boolean flag whether to count it by words or letters:
`[ document, count_letters ]`.

###### count_samplesheet.nf:
```bash
...
// convert each row in the samplesheet into a tuple of File object and a Boolean of count_letter
def get_document_info(LinkedHashMap sample) {
    document  = sample.document ? file(sample.document, checkIfExists: true) : null
    count_letters = (sample.count_letters == "T") || (sample.count_letters == "true") ? true : false   

    return [ document, count_letters ]
}
...
workflow {
     // create a Channel of documents to count either by word or by letter, from the samplesheet
     Channel.fromPath(params.samplesheet).splitCsv(header:true)
            .map { get_document_info(it) }.set { documents_ch }
            
     // launch a sub-workflow of COUNT_DOCUMENT on each document
     COUNT_DOCUMENT(documents_ch)
}
```


##### Define the sub-workflow `COUNT_DOCUMENT` which will process each individual document and count appropriately by its metadata flag:

Instead of refering to a global variable `params.count_letters` as we did previously. 

We refer to the current metadata we are processing: `tuple file(file_in), val(count_letters)`; which is: `[(samplesheet/hello1.txt,F), (samplesheet/hello2.txt,T)]`. And for `T`, we will count by letters; and for `F`, by words.

```bash
process count_document {
  input: 
    tuple file(file_in), val(count_letters)
  output:
    stdout
  
  script:
   if (count_letters) {
    """
    cat ${file_in}
    wc -m ${file_in} | awk '{print \$1}'
    """
   } else {
    """
    cat ${file_in}
    wc -w ${file_in} | awk '{print \$1}'
    """
   }
}

workflow COUNT_DOCUMENT {
    take:
        input_ch

    main:
        count_document(input_ch) | view
}
```

##### Example Outputs:
```bash
N E X T F L O W  ~  version 23.04.2
Launching `workflows/count_samplesheet.nf` [dreamy_fourier] DSL2 - revision: c7971cd787
executor >  local (2)
[3f/9df991] process > COUNT_DOCUMENT:count_document (1) [100%] 2 of 2 ✔
Count Me (using Letters)24

Count Me (using Words)4
```

#### Step 4d. Asynchronous and synchronous job execution (or scatter-gather pattern)

Scatter-gather pattern is an important pattern in pipelining. The concept of this is that we want to split ("scatter") out entries on a fasta file or rows in a samplesheet for parallel execution and on any downstream jobs asynchronously (on as many nodes as possible on `OSCAR`). 

Then in the end of pipeline, we would like to aggregate ("gather") these results; for example, 
 - In RNASeq, to sum up or contrast the gene expressions of the `wild-type` replicates against the `treatment` replicates
 - In metagenomics workflows, to summarize the overall distribution or diversity of all the samples in a data-set.
 
 
Continuing with our toy example of document counting, we now would like to demonstrate this concept by simply gathering all of the individual counts (whether by letter or by words) and summing all of them togethe to give an overall total count. 


###### Add a variable `count_result` which is an synchronous collection/`collect()` of all the scattered or asynchronous runs of `COUNT_DOCUMENT`:

By defining a variable to hold onto `COUNT_DOCUMENT(documents_ch).collect()`, we are essentially assigning `count_results` as the gather-step's result placeholder. In other words, we will wait for all `COUNT_DOCUMENTS` to finish, and all individual document counts to be gathered into `count_results` and then run the final gather step of `sum_all_results(count_results)`. 

###### count_and_summarize.nf:
```bash
process sum_all_results {
  input: 
    path (files)
  output:
    stdout

  script:
    """
    cat ${files} > combined.txt
    awk '{ sum += \$1 } END { print sum }' combined.txt
    """
}

workflow {
    ... 
    // gather all of the results of each document counting sub-workflow
    count_results = COUNT_DOCUMENT(documents_ch).collect()
    
    // and feed this collection of results into a summing function
    sum_all_results(count_results) | view
}
```


##### Example Outputs:
```bash
N E X T F L O W  ~  version 23.04.2
Launching `workflows/count_and_summarize.nf` [zen_baekeland] DSL2 - revision: 478f5774ff
executor >  local (3)
[12/8215ea] process > COUNT_DOCUMENT:count_document (1) [100%] 2 of 2 ✔
[ff/fbd4b5] process > sum_all_results                   [100%] 1 of 1 ✔
28
```

#### Step 4e. Putting it all together in a real-world RNASeq example

We have learned all of the essential patterns involved in creating pipelines; now we can begin to put all of them together in a simple RNASeq pipeline. Below is the code for a full RNASeq pipeline, but don't worry too much about completely understanding all of the steps. 

Our purpose here is to refresh/review the pipeline patterns we just learned applied to a real world example. Each application of the patterns will be called out following the code. 

###### rnaseq_simple.nf:

```bash
workflow PROCESS_SAMPLE {
    take:
        input_ch
        reference_genome

    main:
        fastqc(input_ch)

        trimmed_reads = trimmomatic(input_ch)

        fastqcs = fastqc2(trimmed_reads).collect()
        fastqc_screens = fastq_screen(trimmed_reads).collect()

        aligned_bams = star(trimmed_reads, reference_genome)
        marked_duplicate_bams = mark_duplicate(aligned_bams)

        qualimaps = qualimap(marked_duplicate_bams).collect()

        htseq_counts = htseq_count(marked_duplicate_bams).collect()
        feature_counts = feature_count(marked_duplicate_bams)

        multiqc(fastqcs, fastqc_screens, qualimaps, htseq_counts)
}

// Function to resolve files
def get_sample_info(LinkedHashMap sample) {
    read1  = sample.read1 ? file(sample.read1, checkIfExists: true) : null
    read2 = sample.read2 ? file(sample.read2, checkIfExists: true) : null

    return [ sample.sample_id, read1, read2 ]
}

workflow {
     reference_genome = params.reference_genome

     if (params.reference_genome_fasta != "") {
        reference_genome = build_star_index()
     }

     Channel.fromPath(params.samplesheet).splitCsv(header:true)
            .map { get_sample_info(it) }.set { samples_ch }

     PROCESS_SAMPLE (samples_ch, reference_genome)
}
```

###### Pattern 1: Conditional Logic

If `--reference_genome_fasta` is specified, we will build an index for this reference genome (so that our raw reads could be aligned against this index more efficiently)

```bash
if (params.reference_genome_fasta != "") {
    reference_genome = build_star_index()
}
```

###### Pattern 2: Parsing the Samplesheet
We parse a samplesheet of RNASeq samples which will be `(sample_id, read1, read2)` and carry this metadata throughout the rest of the workflow.

```bash

// Function to resolve files
def get_sample_info(LinkedHashMap sample) {
    read1  = sample.read1 ? file(sample.read1, checkIfExists: true) : null
    read2 = sample.read2 ? file(sample.read2, checkIfExists: true) : null

    return [ sample.sample_id, read1, read2 ]
}

...

Channel.fromPath(params.samplesheet).splitCsv(header:true)
            .map { get_sample_info(it) }.set { samples_ch }
PROCESS_SAMPLE (samples_ch, reference_genome)
```

###### Pattern 3: Scatter-Gather using `collect()`
We will process asynchronously all of the tasks for each RNASeq sample such as:
- QCing the read `fastqc`
- Screening the read for contamination/sequencing artifacts `fastq_screen`
- Aligning the reads and marking duplicates on the aligned reads `star` aligner and `mark_duplicate` tool
- Count the features (individual gene expression) on the algined reads `htseq`

Then finally, we gather all of these results into an awesome data visualization/summarization tool `multiqc`. 

```bash
        fastqcs = fastqc2(trimmed_reads).collect()
        fastqc_screens = fastq_screen(trimmed_reads).collect()

        ...

        qualimaps = qualimap(marked_duplicate_bams).collect()
        htseq_counts = htseq_count(marked_duplicate_bams).collect()

        multiqc(fastqcs, fastqc_screens, qualimaps, htseq_counts)
```

## A Few Closing Thoughts.....

This workshop just provided an introductory overview of running Nextflow worklfows on OSCAR. There is much more customization that you can do and much more advanced things you can perform. Some of these are: 

* Resource allocation by rule (or analysis step) 
* Using a config.yaml file to handle your samples and how you iterate through files
* Running pipelines that skip steps and start at a specific step (for example, pipelines that fail at a certain step, you may not want to repeat everything but instead pick up where you left off)
* Handling log files within specific steps so that you get detailed output for each rule 
* And much more....